In [ ]:
# imports

%matplotlib inline
import networkx as nx
import matplotlib.pyplot as plt
from random import uniform, seed
import numpy as np
import pandas as pd
import random
import time
import csv
import sys

In [ ]:
# mounting Google Drive

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Do not run

# Load your CSV file
input_path = "/content/drive/MyDrive/Final Year Project 2024-25 (Group 1)/Yelp-Reviews-Dataset/yelp_reviews_with_sentiment_(4039).csv"  # Update with your exact path
df = pd.read_csv(input_path)

# Convert the relevant columns to numeric in case of parsing issues
for col in ['Brand A', 'Brand B', 'Brand C']:
    df[col] = pd.to_numeric(df[col], errors='coerce')

# Function to subtract 0.04 from non-zero and clip result
def adjust_value(x):
    if pd.isnull(x):
        return x
    elif x != 0:
        return max(min(x - 0.04, 1), -1)
    else:
        return x

# Apply the adjustment
for col in ['Brand A', 'Brand B', 'Brand C']:
    df[col] = df[col].apply(adjust_value)

# Save the adjusted CSV
output_path = "/content/drive/MyDrive/Final Year Project 2024-25 (Group 1)/Yelp-Reviews-Dataset/yelp_reviews_with_sentiment_adjusted_(4039).csv"
df.to_csv(output_path, index=False)

print(f"Saved adjusted file to: {output_path}")


Saved adjusted file to: /content/drive/MyDrive/Final Year Project 2024-25 (Group 1)/Yelp-Reviews-Dataset/yelp_reviews_with_sentiment_adjusted_(4039).csv


In [ ]:
# Get dataset paths interactively
edge_file = input("Enter the path for the edge list dataset (CSV): ")
node_file = input("Enter the path for the node attributes dataset (CSV): ")

# Initialize directed graph
G = nx.DiGraph()

# Initialize matrices
bias_matrix = np.zeros((6005, 3))
trust_matrix = np.zeros((6005, 6005))

# Step 1: Load Node Attributes First (Ensures all nodes exist in the graph)
print("Loading node attributes first to include isolated nodes...")
start_time = time.perf_counter()

with open(node_file, "r") as csvfile:
    csv_reader = csv.reader(csvfile)
    next(csv_reader)  # Skip header row

    for row in csv_reader:
        node = int(row[1])  # Use the second column (Node ID)
        bias_A = float(row[2])
        bias_B = float(row[3])
        bias_C = float(row[4])

        # Add node to the graph with attributes
        G.add_node(node, bias_A = bias_A, bias_B = bias_B, bias_C = bias_C)

        # Update the bias matrix
        bias_matrix[node - 1] = [bias_A, bias_B, bias_C]

print(f"Total nodes added: {G.number_of_nodes()}")
print(f"Time taken: {(time.perf_counter() - start_time):.2f} seconds")

# Step 2: Load edges from dataset
print("Loading edges into graph...")
start_time = time.perf_counter()

with open(edge_file, "r") as csvfile:
    csv_reader = csv.reader(csvfile)

    for row in csv_reader:
        from_node = int(row[0])
        to_node = int(row[1])
        edge_weight = float(row[2]) / 10

        # Add edge with weight and sign attributes
        G.add_edge(from_node, to_node, weight = edge_weight)

        # Update the trust matrix
        trust_matrix[from_node - 1][to_node - 1] = edge_weight

print(f"Graph loaded with {G.number_of_nodes()} nodes and {G.number_of_edges()} edges.")
print(f"Time taken: {(time.perf_counter() - start_time):.2f} seconds")

Enter the path for the edge list dataset (CSV): /content/drive/MyDrive/Final Year Project 2024-25 (Group 1)/Signed-Bitcoin-Dataset/soc-sign-bitcoinotc.csv
Enter the path for the node attributes dataset (CSV): /content/drive/MyDrive/Final Year Project 2024-25 (Group 1)/Yelp-Reviews-Dataset/yelp_reviews_with_sentiment_adjusted_(6005).csv
Loading node attributes first to include isolated nodes...
Total nodes added: 6005
Time taken: 0.77 seconds
Loading edges into graph...
Graph loaded with 6005 nodes and 35592 edges.
Time taken: 0.69 seconds


In [ ]:
# Pruning the graph to keep only the "strong" edges, both positive and negative

def prune_graph(G):
  H = nx.DiGraph()
  x = 0.3 # can vary this

  # Copy all nodes and their attributes
  H.add_nodes_from(G.nodes(data=True))

  # Add only strong edges
  for u, v, data in G.edges(data=True):
      weight = data.get('weight', 0)
      if weight >= x or weight <= -x:
          H.add_edge(u, v, **data)

  return H

H = prune_graph(G)
print(f"Original Graph: {G.number_of_edges()} edges")
print(f"Pruned Graph: {H.number_of_edges()} edges")

Original Graph: 35592 edges
Pruned Graph: 9199 edges


In [ ]:
# Getting an idea of how many target nodes are present for each brand

def get_target_count(G):
  t1 = 0.35
  t2 = 0.95
  count_A = 0
  count_B = 0
  count_C = 0
  for node in G.nodes():
    bias_A = bias_matrix[node - 1][0]
    bias_B = bias_matrix[node - 1][1]
    bias_C = bias_matrix[node - 1][2]
    if t1 <= bias_A <= t2:
      count_A += 1
    if t1 <= bias_B <= t2:
      count_B += 1
    if t1 <= bias_C <= t2:
      count_C += 1

  return count_A, count_B, count_C

count_A, count_B, count_C = get_target_count(G)
print(f"Target nodes for Brand A: {count_A}")
print(f"Target nodes for Brand B: {count_B}")
print(f"Target nodes for Brand C: {count_C}")

Target nodes for Brand A: 1780
Target nodes for Brand B: 1861
Target nodes for Brand C: 1496


In [ ]:
# Processing the "Gain" attribute (from old paper)

def process_gain(G):
  gain = {}
  check = {}
  spread = []

  for node in G.nodes():
    gain[node] = 0

  for node in G.nodes():
    for nd in G.nodes():
      check[node] = False
    spread.append(node)
    for n in spread:
      if (check[n] == False):
        for succ in G.successors(node):
          amt = (bias_matrix[node - 1][0] * trust_matrix[node - 1][succ - 1] + bias_matrix[succ - 1][0]) / 2
          if (amt >= 0.5):
            spread.append(succ)
        check[n] = True
      gain[node] = len(set(spread))
      spread.clear()

  return gain

In [ ]:
# Processing the other node attributes for seed selection

def get_sorted_node_lists(G):
  # 1. Out-degree
  outdegree_sorted = sorted(G.out_degree(), key=lambda x: x[1], reverse=True)

  # 2. Gain coefficient
  gain_values = process_gain(G)
  gain_values_sorted = sorted(gain_values.items(), key=lambda x: x[1], reverse=True)

  # 3. Number of target successors (bias between t1 and t2)
  t1 = 0.35
  t2 = 0.95
  target_successor_counts = {}
  for node in G.nodes():
      count = 0
      for succ in G.successors(node):
          # bias_matrix[node_id - 1][0] gives bias for Brand A
          # Adjust index based on your bias column if needed
          try:
              bias = bias_matrix[succ - 1][0]  # Assuming node IDs start from 1
              if t1 <= bias <= t2:
                  count += 1
          except IndexError:
              pass  # skip if index is out of bounds
      target_successor_counts[node] = count

  target_successors_sorted = sorted(target_successor_counts.items(), key=lambda x: x[1], reverse=True)

  return outdegree_sorted, gain_values_sorted, target_successors_sorted

out_sorted, gain_sorted, target_sorted = get_sorted_node_lists(H)

print("Top 5 nodes by out-degree:", out_sorted[:5])
print("Top 5 nodes by gain:", gain_sorted[:5])
print("Top 5 nodes by target successors:", target_sorted[:5])

Top 5 nodes by out-degree: [(1810, 211), (2125, 128), (4172, 127), (2642, 117), (2028, 100)]
Top 5 nodes by gain: [(4291, 19), (1018, 17), (35, 15), (832, 13), (1317, 12)]
Top 5 nodes by target successors: [(1810, 69), (2642, 38), (4172, 36), (2067, 35), (2125, 35)]


In [ ]:
# Seed Selection for Brand A (Our Method)

def RiKaAr(K, coeffs):
  t1 = 0.35
  t2 = 0.95

  # Normalize coefficients so they sum to 1
  total = sum(coeffs)
  norm_coeffs = [c / total for c in coeffs]

  # Determine how many nodes to pick from each strategy
  k1 = int(norm_coeffs[0] * K)
  k2 = int(norm_coeffs[1] * K)
  k3 = K - k1 - k2  # Ensure total is exactly K
  print(f"Seeds from strategies: out-degree = {k1}, gain = {k2}, target-successors = {k3}")

  # Initial seed set: nodes with strong positive bias
  seed_set = []
  for node in range(1, len(bias_matrix) + 1):  # assuming nodes are 1-indexed
      if bias_matrix[node - 1][0] >= t2:
          seed_set.append(node)

  seed_set = set(seed_set)  # Use set for fast lookup & uniqueness

  # Helper function to add unique, eligible nodes to seed set
  def add_seeds(sorted_list, count):
      added = 0
      for node, _ in sorted_list:
          if node not in seed_set and bias_matrix[node - 1][0] > 0:
              seed_set.add(node)
              added += 1
          if added == count:
              break

  # Add nodes from each strategy
  add_seeds(out_sorted, k1)
  add_seeds(gain_sorted, k2)
  add_seeds(target_sorted, k3)

  return list(seed_set)

K = int(input("Enter number of seed nodes (K): "))
w1 = float(input("Enter weight for out-degree strategy: "))
w2 = float(input("Enter weight for gain strategy: "))
w3 = float(input("Enter weight for target-successor strategy: "))

coeffs = (w1, w2, w3)
seed_nodes_A = RiKaAr(K, coeffs)

print(f"Total seed nodes selected: {len(seed_nodes_A)}")

Enter number of seed nodes (K): 60
Enter weight for out-degree strategy: 8
Enter weight for gain strategy: 1
Enter weight for target-successor strategy: 1
Seeds from strategies: out-degree = 48, gain = 6, target-successors = 6
Total seed nodes selected: 180


In [ ]:
# Seed Selection for Brand B

def select_seed_set_B(K):
    total_nodes = len(bias_matrix)
    seed_set_1 = set(seed_nodes_A)
    seed_set_2 = set()

    t1 = 0.35
    t2 = 0.95

    # Step 1: Add nodes with bias >= t2 and not in seed_set_1
    for node in range(1, total_nodes + 1):
        if node not in seed_set_1 and bias_matrix[node - 1][1] >= t2:
            seed_set_2.add(node)

    # Step 2: Add top K nodes with highest outdegree with positive bias not in either seed set
    added = 0
    for node, _ in out_sorted:
        if node not in seed_set_1 and node not in seed_set_2 and bias_matrix[node - 1][1] > 0:
            seed_set_2.add(node)
            added += 1
        if added == K:
            break

    return list(seed_set_2)

K = int(input("Enter number of seed nodes (K): "))
seed_nodes_B = select_seed_set_B(K)

print(f"Total seed nodes selected: {len(seed_nodes_B)}")

Enter number of seed nodes (K): 60
Total seed nodes selected: 202


In [ ]:
# Seed Selection for Brand C

def select_seed_set_C(K, G):
    total_nodes = len(bias_matrix)
    seed_set_1 = set(seed_nodes_A)
    seed_set_2 = set(seed_nodes_B)
    seed_set_3 = set()

    t1 = 0.35
    t2 = 0.95

    # Step 1: Add nodes with bias >= t2 and not in seed_set_1 or seed_set_2
    for node in range(1, total_nodes + 1):
        if node not in seed_set_1 and node not in seed_set_2 and bias_matrix[node - 1][2] >= t2:
            seed_set_3.add(node)

    # Step 2: Apply Degree Discount to select K nodes from remaining eligible nodes
    eligible_nodes = [
        node for node in range(1, total_nodes + 1)
        if node not in seed_set_1 and node not in seed_set_2 and node not in seed_set_3 and bias_matrix[node - 1][2] > 0
    ]

    # Create subgraph of only eligible nodes
    subG = G.subgraph(eligible_nodes)

    # Degree Discount Implementation
    degree_count = dict(subG.degree)
    dd = {v: degree_count[v] for v in subG.nodes()}
    tu = {v: 0 for v in subG.nodes()}
    p = 0.1

    selected = []
    for _ in range(min(K, len(subG))):
        if not dd:
            break
        node = max(dd.items(), key=lambda x: x[1])[0]
        selected.append(node)
        dd.pop(node)
        neighbors = set(subG.neighbors(node)) - set(selected)
        for n in neighbors:
            tu[n] += 1
            dd[n] = degree_count[n] - 2 * tu[n] - (degree_count[n] - tu[n]) * tu[n] * p

    seed_set_3.update(selected)

    return list(seed_set_3)

K = int(input("Enter number of seed nodes (K): "))
seed_nodes_C = select_seed_set_C(K, H)

print(f"Total seed nodes selected: {len(seed_nodes_C)}")


Enter number of seed nodes (K): 60
Total seed nodes selected: 171


In [ ]:
# Negative Seed Selection for Brands A, B, and C

def get_negative_seed_sets():
    total_nodes = len(bias_matrix)
    neg_seed_A, neg_seed_B, neg_seed_C = set(), set(), set()

    t1 = 0.35
    t2 = 0.95

    for node in range(1, total_nodes + 1):
        bias_A = bias_matrix[node - 1][0]
        bias_B = bias_matrix[node - 1][1]
        bias_C = bias_matrix[node - 1][2]

        if bias_A <= -t2:
            neg_seed_A.add(node)
        elif bias_B <= -t2:
            neg_seed_B.add(node)
        elif bias_C <= -t2:
            neg_seed_C.add(node)



    return list(neg_seed_A), list(neg_seed_B), list(neg_seed_C)

negative_seeds_A, negative_seeds_B, negative_seeds_C = get_negative_seed_sets()
print(f"Negative seed set for Brand A: {len(negative_seeds_A)} nodes")
print(f"Negative seed set for Brand B: {len(negative_seeds_B)} nodes")
print(f"Negative seed set for Brand C: {len(negative_seeds_C)} nodes")

Negative seed set for Brand A: 21 nodes
Negative seed set for Brand B: 14 nodes
Negative seed set for Brand C: 4 nodes


In [ ]:
print(f"Positive seed set size for A: {len(seed_nodes_A)} nodes")
print(f"Negative seed set size for A: {len(negative_seeds_A)} nodes")
print(f"Positive seed set size for B: {len(seed_nodes_B)} nodes")
print(f"Negative seed set size for B: {len(negative_seeds_B)} nodes")
print(f"Positive seed set size for C: {len(seed_nodes_C)} nodes")
print(f"Negative seed set size for C: {len(negative_seeds_C)} nodes")

Positive seed set size for A: 180 nodes
Negative seed set size for A: 21 nodes
Positive seed set size for B: 202 nodes
Negative seed set size for B: 14 nodes
Positive seed set size for C: 171 nodes
Negative seed set size for C: 4 nodes


In [ ]:
# Function definition of LT model; takes in the graph G and the seed sets Sp and Sn as lists of lists, as parameters

def LT(G, Sp, Sn):
  print("PARALLEL PROPAGATION LT CALLED: ")
  tic = time.time()

  active_pos = [lst.copy() for lst in Sp]
  active_neg = [lst.copy() for lst in Sn]

  IA = {}
  pos_influenced = {}
  neg_influenced = {}
  is_locked = {}
  threshold = {}

  for node in G.nodes:
    IA[node] = [lst.copy() for lst in bias_matrix[node - 1]]
    is_locked[node] = [False, False, False]
    pos_influenced[node] = False
    neg_influenced[node] = False
    for i in range(0, 3):
      if node in active_pos[i]:
        is_locked[node][i] = True
        pos_influenced[node] = True
      if node in active_neg[i]:
        is_locked[node][i] = True
        neg_influenced[node] = True
    threshold[node] = 0.95

  list_pos = [lst.copy() for lst in active_pos]
  list_neg = [lst.copy() for lst in active_neg]

  k = -1
  count = 0
  count_list = []
  ncount = 0
  ncount_list = []

  while (active_pos[0] or active_pos[1] or active_pos[2] or active_neg[0] or active_neg[1] or active_neg[2]):
    k += 1

    token1 = random.randint(0, 2) # represents the competitor
    token2 = random.randint(0, 1) # represents the sign

    if (token2 == 0):
      if (active_pos[token1]):
        s_t = active_pos[token1].pop(0)
        for n in G.neighbors(s_t):
          if (is_locked[n][token1] == False):
            amt = (IA[s_t][token1] * trust_matrix[n - 1][s_t -1]) / (G.in_degree(n))
            if amt >= 0:
              IA[n][token1] += amt
              if (IA[n][token1] >= threshold[n] and pos_influenced[n] == False):
                pos_influenced[n] = True
                is_locked[n][token1] = True
                active_pos[token1].append(n)
                list_pos[token1].append(n)
                count_list.append(n)
                count += 1
            elif amt < 0:
              IA[n][token1] += amt
              if (IA[n][token1] <= (-1 * threshold[n]) and neg_influenced[n] == False):
                neg_influenced[n] = True
                is_locked[n][token1] = True
                active_neg[token1].append(n)
                list_neg[token1].append(n)
                ncount_list.append(n)
                ncount += 1
    else:
      if (active_neg[token1]):
        s_t = active_neg[token1].pop(0)
        for n in G.neighbors(s_t):
          if (is_locked[n][token1] == False):
            amt = (IA[s_t][token1] * trust_matrix[n - 1][s_t -1]) / (G.in_degree(n))
            if amt >= 0:
              IA[n][token1] += amt
              if (IA[n][token1] >= threshold[n] and pos_influenced[n] == False):
                pos_influenced[n] = True
                is_locked[n][token1] = True
                active_pos[token1].append(n)
                list_pos[token1].append(n)
                count_list.append(n)
                count += 1
            elif amt < 0:
              IA[n][token1] += amt
              if (IA[n][token1] <= (-1 * threshold[n]) and neg_influenced[n] == False):
                neg_influenced[n] = True
                is_locked[n][token1] = True
                active_neg[token1].append(n)
                list_neg[token1].append(n)
                ncount_list.append(n)
                ncount += 1

  toc = time.time()
  print(f"Time taken: {(toc - tic):.2f} seconds")
  print(f"Total nodes positively influenced: {count}")
  print(f"Total nodes negatively influenced: {ncount}")

  return list_pos, list_neg

In [ ]:
# Calling the Propagation Model with the Seed Sets

Sp = [seed_nodes_A, seed_nodes_B, seed_nodes_C]
Sn = [negative_seeds_A, negative_seeds_B, negative_seeds_C]

list_pos, list_neg = LT(G, Sp, Sn)

PARALLEL PROPAGATION LT CALLED: 
Time taken: 0.13 seconds
Total nodes positively influenced: 528
Total nodes negatively influenced: 2


In [ ]:
for i in range (0, 3):
  print(f"Positive spread for Brand {chr(65 + i)}: {len(list_pos[i])} nodes")
  print(f"Negative spread for Brand {chr(65 + i)}: {len(list_neg[i])} nodes")

Positive spread for Brand A: 408 nodes
Negative spread for Brand A: 22 nodes
Positive spread for Brand B: 349 nodes
Negative spread for Brand B: 15 nodes
Positive spread for Brand C: 324 nodes
Negative spread for Brand C: 4 nodes


In [ ]:
# Identifying the target nodes influenced

def target_nodes(list_pos, list_neg):
  targets = [[], [], []]

  t1 = 0.35
  t2 = 0.95

  for i in range(0, 3):
    for node in list_pos[i]:
      if bias_matrix[node - 1][i] >= t1 and bias_matrix[node - 1][i] <= t2:
        targets[i].append(node)

  return targets

targets = target_nodes(list_pos, list_neg)

for i in range (0, 3):
  print(f"Target nodes for Brand {chr(65 + i)}: {len(targets[i])} nodes")

Target nodes for Brand A: 286 nodes
Target nodes for Brand B: 204 nodes
Target nodes for Brand C: 209 nodes


In [ ]:
# Implementation of PSO Algorithm

# Parameters for PSO
NUM_PARTICLES = 30
MAX_ITER = 25
W = 0.5       # Inertia weight
C1 = 1.5      # Cognitive component
C2 = 1.5      # Social component

# Placeholder for user inputs
K = int(input("Enter number of seed nodes (K): "))  # number of seeds
w1 = float(input("Enter weight for out-degree strategy: "))
w2 = float(input("Enter weight for gain strategy: "))
w3 = float(input("Enter weight for target-successor strategy: "))
initial_coeffs = (w1, w2, w3)  # starting guess

# Fitness Function
def fitness_function(coeffs):
    # Normalize coefficients
    total = sum(coeffs)
    coeffs = [c / total for c in coeffs]

    # Step 1: Seed selection for Brand A
    seed_nodes_A = RiKaAr(K, coeffs)

    # Step 2: Seed selection for Brands B and C
    seed_nodes_B = select_seed_set_B(K)
    seed_nodes_C = select_seed_set_C(K, H)

    # Step 3: Negative seed selection for all brands
    negative_seeds_A, negative_seeds_B, negative_seeds_Cs = get_negative_seed_sets()

    # Step 4: Influence propagation
    Sp = [seed_nodes_A, seed_nodes_B, seed_nodes_C]
    Sn = [negative_seeds_A, negative_seeds_B, negative_seeds_C]
    list_pos, list_neg = LT(G, Sp, Sn)

    # Step 5: Target node count
    targets = target_nodes(list_pos, list_neg)

    # Fitness calculation
    f = (len(list_pos[0]) - len(list_pos[1]) - len(list_pos[2])) + (len(targets[0]) - len(targets[1]) - len(targets[2]))
    return f

# Particle Swarm Optimization
class Particle:
    def __init__(self):
        self.position = np.random.dirichlet(np.ones(3), size=1)[0]  # random coeffs summing to 1
        self.velocity = np.zeros(3)
        self.best_position = np.copy(self.position)
        self.best_score = float('-inf')

    def update_velocity(self, global_best):
        r1, r2 = random.random(), random.random()
        cognitive = C1 * r1 * (self.best_position - self.position)
        social = C2 * r2 * (global_best - self.position)
        self.velocity = W * self.velocity + cognitive + social

    def update_position(self):
        self.position = self.position + self.velocity
        # Ensure non-negative and normalize
        self.position = np.maximum(self.position, 0)
        if self.position.sum() == 0:
            self.position = np.array([1/3, 1/3, 1/3])
        else:
            self.position = self.position / self.position.sum()

# PSO main loop
def pso_optimize():
    swarm = [Particle() for _ in range(NUM_PARTICLES)]
    global_best = None
    global_best_score = float('-inf')

    for iteration in range(MAX_ITER):
        print(f"Iteration {iteration+1}/{MAX_ITER}")
        for particle in swarm:
            score = fitness_function(particle.position)
            if score > particle.best_score:
                particle.best_score = score
                particle.best_position = np.copy(particle.position)
            if score > global_best_score:
                global_best_score = score
                global_best = np.copy(particle.position)

        for particle in swarm:
            particle.update_velocity(global_best)
            particle.update_position()

    print(f"Best Coefficients: {global_best}, Best Fitness: {global_best_score}")
    return global_best

# Example usage
best_coeffs = pso_optimize()


Enter number of seed nodes (K): 60
Enter weight for out-degree strategy: 8
Enter weight for gain strategy: 1
Enter weight for target-successor strategy: 1
Iteration 1/25
Seeds from strategies: out-degree = 51, gain = 5, target-successors = 4
PARALLEL PROPAGATION LT CALLED: 
Time taken: 0.12 seconds
Total nodes positively influenced: 529
Total nodes negatively influenced: 2
Seeds from strategies: out-degree = 51, gain = 6, target-successors = 3
PARALLEL PROPAGATION LT CALLED: 
Time taken: 0.12 seconds
Total nodes positively influenced: 529
Total nodes negatively influenced: 2
Seeds from strategies: out-degree = 25, gain = 25, target-successors = 10
PARALLEL PROPAGATION LT CALLED: 
Time taken: 0.13 seconds
Total nodes positively influenced: 522
Total nodes negatively influenced: 2
Seeds from strategies: out-degree = 7, gain = 45, target-successors = 8
PARALLEL PROPAGATION LT CALLED: 
Time taken: 0.12 seconds
Total nodes positively influenced: 510
Total nodes negatively influenced: 2
Seed